# CAE-Transformer flow field prediction model

## Environment Setup

This notebook requires **MindSpore version >= 2.0.0** to support new APIs including: *mindspore.jit, mindspore.jit_class, mindspore.data_sink*. Please check [MindSpore Installation](https://www.mindspore.cn/install/en) for details.

In addition, **MindFlow version >=0.1.0** is also required. If it has not been installed in your environment, please select the right version and hardware, then install it as follows.

In [ ]:
mindflow_version = "0.1.0"  # update if needed
# GPU Comment out the following code if you are using NPU.
!pip uninstall -y mindflow-gpu
!pip install mindflow-gpu==$mindflow_version

# NPU Uncomment if needed.
# !pip uninstall -y mindflow-ascend
# !pip install mindflow-ascend==$mindflow_version

## Introduction

In order to effectively reduce the design cost and cycle time of using CFD methods, the reduced-order model (ROM) has gained wide attention in recent years. For complex compressible flows, using linear methods such as Proper Orthogonal Decomposition (POD) for flow field dimensionality reduction requires a large number of modes to ensure the reconstruction accuracy. It has been shown that the modes number can be effectively reduced by using nonlinear dimensionality reduction methods. Convolutional Autoencoder (CAE) is a kind of neural network composed of encoder and decoder, which can realize data dimensionality reduction and recon-struction, and can be regarded as a nonlinear extension of POD method. CAE is used for nonlinear dimension-ality reduction, and Transformer is used for time evolution. The CAE-Transformer can obtain high reconstruction and prediction accuracy on the premise of using less latents for unsteady compressible flows.

### Framework of CAE-Transformer

The basic framework of CAE-Transformer is mainly based on [paper1](https://doi.org/10.13700/j.bh.1001-5965.2022.0085) and [paper2](https://doi.org/10.1609/aaai.v35i12.17325). It consists of CAE and Transformer, where the encoder in CAE reduces the dimensionality of the time series flow field to achieve feature extraction, Transformer learns low dimensional spatiotemporal features and makes predictions, and the decoder in CAE realizes flow field reconstruction:

+ Input：Input the flow field for a period of time;

+ Compression：Extract high-dimensional spatiotemporal flow characteristics by dimensionality reduction of the flow field using the encoder of CAE;

+ Evolution：Learning the evolution of spatiotemporal characteristics of low dimensional spatial flow fields through Transformer and predicting the next moment;

+ Reconstruction：Restore the predicted low-dimensional features of the flow field to high-dimensional space through the decoder of CAE;

+ Output：Output the predicted results of the transient flow field at the next moment.

![CAE-Transformer.png](./images/cae_transformer_structure.png)

### Dataset

**Source**: Numerical simulation flow field data of two-dimensional flow around a cylinder, provided by the team of Associate Professor Yu Jian, School of Aeronautical Science and Engineering, Beijing University of Aeronautics and Astronautics.

**Generation**: The calculation status and establishment method of the dataset of two-dimensional cylindrical flow are described in the paper.

**Format**: The data set is numerically simulated for the flow around a cylinder with 10 Reynolds numbers. The flow field data at each Reynolds number contains 401 time steps, and the flow field data at each time step is a 256*256 two-dimensional flow field. The data type of each variable is float32, and the total size of the dataset is about 1.96GB.

**Link**: [2D_cylinder_flow.npy](https://download.mindspore.cn/mindscience/mindflow/dataset/applications/data_driven/cae-transformer/2D_cylinder_flow.npy)

## Preparation works

Import the library required for training, where the src folder includes dataset processing functions, network models, and training loss visualization functions.

The training defaults to the dynamic graph mode (PYNATIVE) of the Mindspore framework, and trains on the GPU (default) or Ascend (single card).

In [ ]:
import os
import time
import argparse
import yaml
import numpy as np

from mindspore import nn, ops, context, save_checkpoint, set_seed, jit, data_sink
from src import create_caetransformer_dataset, plot_train_loss, CaeInformer
from eval import cae_transformer_prediction, cae_transformer_eval

np.random.seed(0)
set_seed(0)
parser = argparse.ArgumentParser(description="CAE-Transformer for 2D cylinder flow")
parser.add_argument(
    "--mode",
    type=str,
    default="PYNATIVE",
    choices=["GRAPH", "PYNATIVE"],
    help="Context mode, support 'GRAPH', 'PYNATIVE'",
)
parser.add_argument(
    "--save_graphs",
    type=bool,
    default=False,
    choices=[True, False],
    help="Whether to save intermediate compilation graphs",
)
parser.add_argument("--save_graphs_path", type=str, default="./graphs")
parser.add_argument(
    "--device_target",
    type=str,
    default="GPU",
    choices=["GPU", "Ascend"],
    help="The target device to run, support 'Ascend', 'GPU'",
)
parser.add_argument(
    "--device_id", type=int, default=0, help="ID of the target device"
)
parser.add_argument("--config_file_path", type=str, default="./config.yaml")
args = parser.parse_args()
context.set_context(mode=context.GRAPH_MODE if args.mode.upper().startswith("GRAPH") else context.PYNATIVE_MODE,
                    save_graphs=args.save_graphs,
                    save_graphs_path=args.save_graphs_path,
                    device_target=args.device_target,
                    device_id=args.device_id)

## Read config

Import the corresponding dataset, CAE model, and optimizer parameter configuration from the `config.yaml` file.

In [ ]:
config_file_path = "./config.yaml"
# prepare params
with open(config_file_path, 'r') as f:
    config = yaml.safe_load(f)
data_params = config["data"]
model_params = config["cae_transformer"]
optimizer_params = config["optimizer"]

# prepare summary file
summary_dir = optimizer_params["summary_dir"]
ckpt_dir = os.path.join(summary_dir, "ckpt")

## Model initialization

Initialize the model according to the configuration in config.yaml, including CAE and Transformer networks.

Use MSE loss function and Adam optimizer.

In [ ]:
# prepare model
model = CaeInformer(**model_params)
loss_fn = nn.MSELoss()
optimizer = nn.AdamWeightDecay(
    model.trainable_params(),
    optimizer_params["lr"],
    weight_decay=optimizer_params["weight_decay"],
)

def forward_fn(data, label):
    logits = model(data)
    loss = loss_fn(logits, label)
    return loss

time_now = time.time()
model.set_train()

grad_fn = ops.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=False)

@jit
def train_step(data, label):
    loss, grads = grad_fn(data, label)
    loss = ops.depend(loss, optimizer(grads))
    return loss

## Construct dataset

Construct a CAE-Transforme dataset according to the data path read in config.yaml, and do data sinking.

The link of the dataset is : [2D_cylinder_flow.npy](https://download.mindspore.cn/mindscience/mindflow/dataset/applications/data_driven/cae-transformer/2D_cylinder_flow.npy).

In [ ]:
# prepare dataset
dataset, eval_data = create_caetransformer_dataset(
    data_params['data_path'],
    data_params["batch_size"],
    data_params["seq_len"],
    data_params["pred_len"],
)

# data sink
sink_process = data_sink(train_step, dataset, sink_size=1)
train_data_size = dataset.get_dataset_size()

## Training

With version MindSpore >= 2.0.0, neural networks can be trained using the functional programming paradigm.

In [ ]:
print(f"====================Start cae transformer train=======================")
train_loss = []
model.set_train()
for epoch in range(1, optimizer_params["epochs"] + 1):
    local_time_beg = time.time()
    epoch_train_loss = 0
    for _ in range(train_data_size):
        epoch_train_loss = ops.squeeze(sink_process(), axis=())
    train_loss.append(epoch_train_loss)
    print(f"epoch: {epoch} train loss: {epoch_train_loss} epoch time: {time.time() - local_time_beg:.2f}s")

    if epoch % optimizer_params["save_ckpt_interval"] == 0:
        save_checkpoint(model, f"{ckpt_dir}/model_{epoch}.ckpt")
    if epoch % optimizer_params["eval_interval"] == 0:
        model.set_train(False)
        cae_transformer_eval(model, eval_data, data_params)
        model.set_train(True)
print(f"=====================End cae train========================")
plot_train_loss(train_loss, summary_dir, optimizer_params["epochs"], "cae")
cae_transformer_prediction(args)

====================Start cae transformer train=====================
epoch: 1 train loss: 1.6767721 epoch time: 20.05s
epoch: 2 train loss: 1.3125554 epoch time: 11.43s
epoch: 3 train loss: 1.0195727 epoch time: 11.36s
epoch: 4 train loss: 0.78560066 epoch time: 11.45s
epoch: 6 train loss: 0.45436704 epoch time: 11.63s
epoch: 7 train loss: 0.34071732 epoch time: 11.28s
epoch: 8 train loss: 0.25292587 epoch time: 11.78s
epoch: 9 train loss: 0.18576089 epoch time: 11.50s
epoch: 10 train loss: 0.13489997 epoch time: 11.36s
=================Start cae-transformer evaluation=====================
eval loss: 0.1344970128662193
===================End transformer evaluation====================
epoch: 11 train loss: 0.09680262 epoch time: 11.57s
epoch: 12 train loss: 0.06859333 epoch time: 11.35s
epoch: 13 train loss: 0.04796055 epoch time: 11.57s
epoch: 14 train loss: 0.03306444 epoch time: 11.67s
epoch: 15 train loss: 0.022457568 epoch time: 11.77s
epoch: 16 train loss: 0.0150149 epoch time: 10

## Prediction result

The following is a comparison of CAE-Transformer and the real value:

<figure class="harf">
    <img src="./images/prediction_result.gif" title="prediction result" width="500"/>
</figure>

The results show the velocity of different locations in the flow field over time. The average relative error between the predicted results and the true values is 6.3e-06